In [1]:
from mwviews.api import PageviewsClient
import mwapi
import wikitools
import pandas as pd
import time


my_agent = 'mwapi testing <p.gildersleve@lse.ac.uk>'
# session = mwapi.Session('https://en.wikipedia.org',
#                         formatversion=2,
#                         user_agent=my_agent)
async_session = mwapi.AsyncSession('https://en.wikipedia.org',
                    formatversion=2, user_agent=my_agent)


In [40]:
toparts = pd.read_csv('data/topviews-2024_07_31.csv')
artlist = toparts['Page'].unique().tolist() # ~1000 top articles yesterday

In [138]:
batchsize = 200
net_articles = set(artlist)
net_edges = set()
n = 0
while n < len(artlist):
    print(f'Getting links for {n} to {n+batchsize}')
    try:
        arts = artlist[n:n+batchsize]
        res = await wikitools.pipeline_get_links('en.wikipedia', my_agent,
                                                titles=arts,
                                                gl_args={'mode':['out', 'in'],
                                                        'resolve_redirects':False})
        res_articles = set([x['title'] for y in res['links']['in'].values() for x in y] +
                [x['title'] for y in res['links']['out'].values() for x in y])
        res_edges = set([(k, x['title']) for k, v in res['links']['out'].items() for x in v] +
                    [(x['title'], k) for k, v in res['links']['in'].items() for x in v])
        net_articles.update(res_articles)
        net_edges.update(res_edges)
        n += batchsize
        if batchsize <= 100:
            batchsize = batchsize * 2
    except ValueError as v: # handle rate error, this doesn't all run automatically, so need to address further issues
        print('ValueError', v)
        #split arts in half and try again
        batchsize = batchsize // 2
        time.sleep(10)
    except ConnectionError as v: # handle rate error, this doesn't all run automatically, so need to address further issues
        print('ValueError', v)
        #split arts in half and try again
        batchsize = batchsize // 2
        time.sleep(10)

Getting links for 0 to 200


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x22cc31110>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x21a5a1d30>, 20353.139497756)]']
connector: <aiohttp.connector.TCPConnector object at 0x17852e590>


Getting links for 200 to 400


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1cf920850>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1f4e771c0>, 20505.162569126), (<aiohttp.client_proto.ResponseHandler object at 0x1f4e89470>, 20514.987242265)]']
connector: <aiohttp.connector.TCPConnector object at 0x1bc097690>


Getting links for 400 to 600


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1a1df7690>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x22743a9e0>, 20729.70512042)]']
connector: <aiohttp.connector.TCPConnector object at 0x1a1694790>


Getting links for 600 to 800


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1d35e7690>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1f85a70e0>, 20901.315460626)]']
connector: <aiohttp.connector.TCPConnector object at 0x1c591c790>


Getting links for 800 to 1000


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x231daf690>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1a5eed160>, 20988.926724434), (<aiohttp.client_proto.ResponseHandler object at 0x19608bc40>, 20999.859092153)]']
connector: <aiohttp.connector.TCPConnector object at 0x252438f10>


In [139]:
print(len(net_articles))
print(len(net_edges))
edges_series = pd.DataFrame(net_edges, columns=['source', 'target'])
edges_series

2666267
6180844


,source,target
0,History of Ohio University,George Washington
1,Veeru Dada,India
2,Pedro Pascal,Buffy the Vampire Slayer
3,Arndtsche Kaffeeaufgußmaschine,English language
4,Tennis at the 2001 Summer Universiade,Germany
...,...,...
6180839,Takashi Amano,Japan
6180840,Taroko people,Taiwan
6180841,Clarisse Moh,France
6180842,Kiki Bertens,Coco Gauff


In [134]:
# get ordered list - hoping to make collection of articles more efficient for subsequent API calls
# not sure if / how much this will actually help

top_indegree = edges_series['target'].value_counts()
ordered_artlist = ([x for x in top_indegree.index if x in artlist] +
                   [x for x in artlist if x not in top_indegree.index])

In [141]:
top_indegree.index[:20]

Index(['United States', 'France', 'Germany', 'United Kingdom', 'India',
       'World War II', 'Australia', 'Japan', 'Canada', 'Russia',
       'New York City', 'China', 'Netherlands', 'Switzerland', 'Iran',
       'English language', 'Turkey', 'World War I', 'Soviet Union',
       'New Zealand'],
      dtype='object', name='target')

In [136]:
batchsize = 5
new_net_articles = set(artlist)
new_net_edges = set()
n = 0
while n < len(ordered_artlist):
    print(f'Getting links for {n} to {n+batchsize}')
    try:
        arts = ordered_artlist[n:n+batchsize]
        res = await wikitools.pipeline_get_links('en.wikipedia', my_agent,
                                                titles=arts,
                                                gl_args={'mode':['out', 'in'],
                                                        'resolve_redirects':False})
        res_articles = set([x['title'] for y in res['links']['in'].values() for x in y] +
                [x['title'] for y in res['links']['out'].values() for x in y])
        res_edges = set([(k, x['title']) for k, v in res['links']['out'].items() for x in v] +
                    [(x['title'], k) for k, v in res['links']['in'].items() for x in v])
        new_net_articles.update(res_articles)
        new_net_edges.update(res_edges)
        n += batchsize
        batchsize = batchsize * 2
    except ValueError as v: # handle rate error, this doesn't all run automatically, so need to address further issues
        print('ValueError', v)
        #split arts in half and try again
        batchsize = batchsize // 2
        time.sleep(10)
    except ConnectionError as v: # handle rate error, this doesn't all run automatically, so need to address further issues
        print('ValueError', v)
        #split arts in half and try again
        batchsize = batchsize // 2
        time.sleep(10)

    

Getting links for 0 to 5


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1fc55f690>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1b627f710>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1fb1711d0>, 18785.999307419)]']
connector: <aiohttp.connector.TCPConnector object at 0x1c8d01fd0>


Getting links for 5 to 15


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1da737690>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x21c3b4130>, 18904.191849593), (<aiohttp.client_proto.ResponseHandler object at 0x1ed032c80>, 18905.925557201), (<aiohttp.client_proto.ResponseHandler object at 0x234775ef0>, 18907.692467689), (<aiohttp.client_proto.ResponseHandler object at 0x222aff4d0>, 18913.249918492), (<aiohttp.client_proto.ResponseHandler object at 0x237347d20>, 18913.941970846)]']
connector: <aiohttp.connector.TCPConnector object at 0x1fc5fca10>


Getting links for 15 to 35


...

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x2385cca10>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1ef963ee0>, 19049.040322109), (<aiohttp.client_proto.ResponseHandler object at 0x1ef961b70>, 19049.040743144), (<aiohttp.client_proto.ResponseHandler object at 0x1ef96eba0>, 19049.057519335), (<aiohttp.client_proto.ResponseHandler object at 0x23398b850>, 19049.05806734), (<aiohttp.client_proto.ResponseHandler object at 0x231305cc0>, 19049.070568246), (<aiohttp.client_proto.ResponseHandler object at 0x21b89a430>, 19049.070855034), (<aiohttp.client_proto.ResponseHandler object at 0x1a27f44b0>, 19049.07118944), (<aiohttp.client_proto.ResponseHandler object at 0x239170b40>, 19049.075965595), (<aiohttp.client_proto.ResponseHandler object at 0x20a6ef3f0>, 19049.076201209), (<aiohttp.client_proto.ResponseHandler object at 0x2312d8280>, 19049.076426972), (<aiohttp.client_proto.ResponseHandler object at 0x2339b56a0>, 1

Getting links for 475 to 795


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x224372910>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1ebf789f0>, 19063.65162414), (<aiohttp.client_proto.ResponseHandler object at 0x1ebf7b230>, 19063.652255531), (<aiohttp.client_proto.ResponseHandler object at 0x2144a69e0>, 19063.652392848), (<aiohttp.client_proto.ResponseHandler object at 0x2144a5470>, 19063.653117559), (<aiohttp.client_proto.ResponseHandler object at 0x22852f310>, 19063.653235568), (<aiohttp.client_proto.ResponseHandler object at 0x226845ef0>, 19063.654262936), (<aiohttp.client_proto.ResponseHandler object at 0x2144a59b0>, 19063.659620733), (<aiohttp.client_proto.ResponseHandler object at 0x2144a5fd0>, 19063.662982257), (<aiohttp.client_proto.ResponseHandler object at 0x2144a6190>, 19063.670254458), (<aiohttp.client_proto.ResponseHandler object at 0x2268477e0>, 19063.674772513), (<aiohttp.client_proto.ResponseHandler object at 0x23c8ef620>, 

Getting links for 795 to 1435


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x21dac9950>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1eea1d400>, 19067.821944557), (<aiohttp.client_proto.ResponseHandler object at 0x2144a6120>, 19067.830925248), (<aiohttp.client_proto.ResponseHandler object at 0x1e8f3b2a0>, 19067.832219652), (<aiohttp.client_proto.ResponseHandler object at 0x2144a6c10>, 19067.839466638), (<aiohttp.client_proto.ResponseHandler object at 0x2339b48a0>, 19067.86667287), (<aiohttp.client_proto.ResponseHandler object at 0x21fe74600>, 19067.869169403), (<aiohttp.client_proto.ResponseHandler object at 0x21a43c980>, 19067.880609789), (<aiohttp.client_proto.ResponseHandler object at 0x21a43e0b0>, 19067.882763363), (<aiohttp.client_proto.ResponseHandler object at 0x21a43dfd0>, 19067.888435435), (<aiohttp.client_proto.ResponseHandler object at 0x21a43dc50>, 19067.888722996), (<aiohttp.client_proto.ResponseHandler object at 0x21a43c280>, 

In [137]:
print(len(new_net_articles))
print(len(new_net_edges))
new_edges_series = pd.DataFrame(new_net_edges, columns=['source', 'target'])
new_edges_series

2666259
6180811


,source,target
0,History of Ohio University,George Washington
1,Veeru Dada,India
2,Pedro Pascal,Buffy the Vampire Slayer
3,Arndtsche Kaffeeaufgußmaschine,English language
4,Tennis at the 2001 Summer Universiade,Germany
...,...,...
6180806,Takashi Amano,Japan
6180807,Taroko people,Taiwan
6180808,Clarisse Moh,France
6180809,Kiki Bertens,Coco Gauff
